In [2]:
# import libraries
import os
import pandas as pd
import json
import gzip
import fasttext 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
import re
import numpy as np
import sklearn

In [64]:
small = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/CSV/medium_tables_seq.csv')

In [65]:
small

,Unnamed: 0,text,name,class
0,0,Heavenly Sentiments Spray Peaceful White Lilie...,name,Product
1,1,"{'pricecurrency': 'USD', 'lowprice': '202.95',...",offers,Product
2,2,"{'reviewcount': '4', 'ratingvalue': '5.00'},{'...",aggregaterating,Product
3,3,"[{'author': 'from -', 'description': 'Arrangem...",review,Product
4,4,NewCondition NewCondition NewCondition NewCond...,itemcondition,Product
...,...,...,...,...
21452,21452,5.0 cu. ft. Smart Front Load Washer with Load ...,name,Product
21453,21453,"{'pricecurrency': 'USD', 'price': '1349.99', '...",offers,Product
21454,21454,"{'name': 'Whirlpool'},{'name': 'GE Appliances'...",brand,Product
21455,21455,"{'reviewcount': '7', 'itemreviewed': '5.0 cu. ...",aggregaterating,Product


In [66]:
small.drop(small[(small['name']=='inalbum') &(small['class']=='MusicRecording')].index, inplace=True)
small.drop(small[(small['name']=='performer') &(small['class']=='Event')].index, inplace=True)
small.drop(small[(small['name']=='eventattendancemode') &(small['class']=='Event')].index, inplace=True)
small.drop(small[(small['name']=='eventstatus') &(small['class']=='Event')].index, inplace=True)
small.drop(small[(small['name']=='owns') &(small['class']=='LocalBusiness')].index, inplace=True)
small["label_complete"] = small["name"] + small["class"]
num_columns = len(set.union(set(small.label_complete.unique())))
small

,Unnamed: 0,text,name,class,label_complete
0,0,Heavenly Sentiments Spray Peaceful White Lilie...,name,Product,nameProduct
1,1,"{'pricecurrency': 'USD', 'lowprice': '202.95',...",offers,Product,offersProduct
2,2,"{'reviewcount': '4', 'ratingvalue': '5.00'},{'...",aggregaterating,Product,aggregateratingProduct
3,3,"[{'author': 'from -', 'description': 'Arrangem...",review,Product,reviewProduct
4,4,NewCondition NewCondition NewCondition NewCond...,itemcondition,Product,itemconditionProduct
...,...,...,...,...,...
21452,21452,5.0 cu. ft. Smart Front Load Washer with Load ...,name,Product,nameProduct
21453,21453,"{'pricecurrency': 'USD', 'price': '1349.99', '...",offers,Product,offersProduct
21454,21454,"{'name': 'Whirlpool'},{'name': 'GE Appliances'...",brand,Product,brandProduct
21455,21455,"{'reviewcount': '7', 'itemreviewed': '5.0 cu. ...",aggregaterating,Product,aggregateratingProduct


In [68]:
test = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/test_tables_seq_9000.csv')

In [69]:
test

,Unnamed: 0,text,name,class
0,0,Plauen | Indonesien Freiburg | Iraq-Kurdistan ...,name,Event
1,1,2022-03-27T16:30:00+01:00 2020-11-15T22:00:00+...,enddate,Event
2,2,"{'name': 'Abenteuer Arena'},{'name': 'Paulussa...",location,Event
3,3,"{'name': 'Freiträumer Festival'},{'name': 'MUN...",organizer,Event
4,4,Máy làm kem Sunny Way RB1119A TỦ MÁT 2 CÁNH KÍ...,name,Product
...,...,...,...,...
48530,48530,"{'name': 'Virtually'},{'name': 'Virtually'},{'...",location,Event
48531,48531,{'email': 'c&#104;i&#108;&#100;&#114;&#101;n&#...,organizer,Event
48532,48532,Marina Olivier Celia Tatjana Valentine Gaia Ré...,givenname,Person
48533,48533,Shunina de LAGARDE van der Beek Mavrenko BOITE...,familyname,Person


In [70]:
test.drop(test[(test['name']=='inalbum') &(test['class']=='MusicRecording')].index, inplace=True)
test.drop(test[(test['name']=='performer') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventattendancemode') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventstatus') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='owns') &(test['class']=='LocalBusiness')].index, inplace=True)
test["label_complete"] = test["name"] + test["class"]
num_columns = len(set.union(set(test.label_complete.unique())))
test

,Unnamed: 0,text,name,class,label_complete
0,0,Plauen | Indonesien Freiburg | Iraq-Kurdistan ...,name,Event,nameEvent
1,1,2022-03-27T16:30:00+01:00 2020-11-15T22:00:00+...,enddate,Event,enddateEvent
2,2,"{'name': 'Abenteuer Arena'},{'name': 'Paulussa...",location,Event,locationEvent
3,3,"{'name': 'Freiträumer Festival'},{'name': 'MUN...",organizer,Event,organizerEvent
4,4,Máy làm kem Sunny Way RB1119A TỦ MÁT 2 CÁNH KÍ...,name,Product,nameProduct
...,...,...,...,...,...
48530,48530,"{'name': 'Virtually'},{'name': 'Virtually'},{'...",location,Event,locationEvent
48531,48531,{'email': 'c&#104;i&#108;&#100;&#114;&#101;n&#...,organizer,Event,organizerEvent
48532,48532,Marina Olivier Celia Tatjana Valentine Gaia Ré...,givenname,Person,givennamePerson
48533,48533,Shunina de LAGARDE van der Beek Mavrenko BOITE...,familyname,Person,familynamePerson


In [71]:
test.drop(['Unnamed: 0','class'], axis=1, inplace=True)
test['text'] = test.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))

In [67]:
small.drop(['Unnamed: 0','class'], axis=1, inplace=True)
small['text'] = small.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))

In [72]:
labels_new = {'worstratingProduct': 0,
 'acceptsreservationsRestaurant': 1,
 'additionalnamePerson': 2,
 'additionalpropertyProduct': 3,
 'addressHotel': 4,
 'addressLocalBusiness': 5,
 'addressPerson': 6,
 'addressProduct': 7,
 'addressRestaurant': 8,
 'addresslocalityLocalBusiness': 9,
 'addressregionLocalBusiness': 10,
 'affiliationPerson': 11,
 'aggregateratingBook': 12,
 'aggregateratingCreativeWork': 13,
 'aggregateratingHotel': 14,
 'aggregateratingLocalBusiness': 15,
 'aggregateratingProduct': 16,
 'aggregateratingRecipe': 17,
 'aggregateratingRestaurant': 18,
 'alternatenameProduct': 19,
 'alternativeheadlineCreativeWork': 20,
 'areaservedLocalBusiness': 21,
 'articlebodyCreativeWork': 22,
 'audienceProduct': 23,
 'authorBook': 24,
 'authorCreativeWork': 25,
 'authorProduct': 26,
 'authorRecipe': 27,
 'availabilityProduct': 28,
 'bestratingProduct': 29,
 'birthdatePerson': 30,
 'birthplacePerson': 31,
 'bookeditionBook': 32,
 'bookformatBook': 33,
 'brandProduct': 34,
 'breadcrumbProduct': 35,
 'byartistMusicAlbum': 36,
 'byartistMusicRecording': 37,
 'categoryProduct': 38,
 'citystatezipLocalBusiness': 39,
 'colorProduct': 40,
 'commentcountCreativeWork': 41,
 'conditionProduct': 42,
 'contactpointLocalBusiness': 43,
 'contactpointPerson': 44,
 'cookingmethodRecipe': 45,
 'cooktimeRecipe': 46,
 'copyrightholderCreativeWork': 47,
 'copyrightyearCreativeWork': 48,
 'creatorCreativeWork': 49,
 'datecreatedCreativeWork': 50,
 'datecreatedMusicRecording': 51,
 'datemodifiedCreativeWork': 52,
 'datemodifiedRecipe': 53,
 'datepublishedBook': 54,
 'datepublishedCreativeWork': 55,
 'datepublishedMusicRecording': 56,
 'datepublishedProduct': 57,
 'datepublishedRecipe': 58,
 'deathdatePerson': 59,
 'depthProduct': 60,
 'disambiguatingdescriptionProduct': 61,
 'doortimeEvent': 62,
 'durationEvent': 63,
 'durationMusicRecording': 64,
 'emailHotel': 65,
 'emailLocalBusiness': 66,
 'emailPerson': 67,
 'enddateEvent': 68,
 'episodenumberTVEpisode': 69,
 'worksforPerson': 70,
 'familynamePerson': 71,
 'faxnumberLocalBusiness': 72,
 'faxnumberPerson': 73,
 'genderPerson': 74,
 'genreBook': 75,
 'genreCreativeWork': 76,
 'genreMusicRecording': 77,
 'geoHotel': 78,
 'geoLocalBusiness': 79,
 'geoPlace': 80,
 'geoProduct': 81,
 'geoRestaurant': 82,
 'givennamePerson': 83,
 'gtin12Product': 84,
 'gtin13Product': 85,
 'gtin14Product': 86,
 'gtin8Product': 87,
 'gtinProduct': 88,
 'hasmapLocalBusiness': 89,
 'hasmenuRestaurant': 90,
 'headlineCreativeWork': 91,
 'headlineRecipe': 92,
 'heightPerson': 93,
 'heightProduct': 94,
 'homelocationPerson': 95,
 'identifierProduct': 96,
 'worklocationPerson': 97,
 'ingredientsRecipe': 98,
 'inlanguageBook': 99,
 'inlanguageCreativeWork': 100,
 'interactioncountCreativeWork': 101,
 'interactionstatisticCreativeWork': 102,
 'interactiontypeCreativeWork': 103,
 'isbnBook': 104,
 'ispartofRecipe': 105,
 'isrelatedtoProduct': 106,
 'issimilartoProduct': 107,
 'itemconditionProduct': 108,
 'itemlistelementProduct': 109,
 'jobtitlePerson': 110,
 'keywordsCreativeWork': 111,
 'keywordsRecipe': 112,
 'knowslanguagePerson': 113,
 'legalnameLocalBusiness': 114,
 'locationEvent': 115,
 'locationLocalBusiness': 116,
 'mainentityofpageCreativeWork': 117,
 'weightProduct': 118,
 'mainentityofpageProduct': 119,
 'mainentityofpageRecipe': 120,
 'makesofferPerson': 121,
 'manufacturerProduct': 122,
 'materialProduct': 123,
 'memberofPerson': 124,
 'menuRestaurant': 125,
 'modelProduct': 126,
 'mpnProduct': 127,
 'nameBook': 128,
 'nameCreativeWork': 129,
 'nameEvent': 130,
 'nameHotel': 131,
 'nameLocalBusiness': 132,
 'nameMusicAlbum': 133,
 'nameMusicRecording': 134,
 'namePlace': 135,
 'nameProduct': 136,
 'nameRecipe': 137,
 'nameRestaurant': 138,
 'nameTVEpisode': 139,
 'nationalityPerson': 140,
 'numberofpagesBook': 141,
 'numtracksMusicAlbum': 142,
 'nutritionRecipe': 143,
 'weightPerson': 144,
 'offersBook': 145,
 'offersCreativeWork': 146,
 'offersProduct': 147,
 'openinghoursLocalBusiness': 148,
 'openinghoursRestaurant': 149,
 'openinghoursspecificationLocalBusiness': 150,
 'openinghoursspecificationPlace': 151,
 'openinghoursspecificationRestaurant': 152,
 'organizerEvent': 153,
 'partofseriesTVEpisode': 154,
 'paymentacceptedLocalBusiness': 155,
 'widthProduct': 156,
 'performersEvent': 157,
 'performtimeRecipe': 158,
 'postalcodeLocalBusiness': 159,
 'preptimeRecipe': 160,
 'priceProduct': 161,
 'pricecurrencyProduct': 162,
 'pricerangeHotel': 163,
 'pricerangeLocalBusiness': 164,
 'pricerangeRestaurant': 165,
 'publisherBook': 166,
 'publisherCreativeWork': 167,
 'publisherRecipe': 168,
 'ratingvalueProduct': 169,
 'recipecategoryRecipe': 170,
 'recipecuisineRecipe': 171,
 'recipeingredientRecipe': 172,
 'recipeinstructionsRecipe': 173,
 'recipeyieldRecipe': 174,
 'releasedateProduct': 175,
 'reviewLocalBusiness': 176,
 'reviewProduct': 177,
 'reviewRecipe': 178,
 'reviewcountProduct': 179,
 'reviewsProduct': 180,
 'sameasLocalBusiness': 181,
 'sameasPerson': 182,
 'sameasPlace': 183,
 'servescuisineRestaurant': 184,
 'shop-currencyProduct': 185,
 'starratingHotel': 186,
 'streetaddressLocalBusiness': 187,
 'suitablefordietRecipe': 188,
 'telephoneHotel': 189,
 'telephoneLocalBusiness': 190,
 'telephonePerson': 191,
 'telephonePlace': 192,
 'telephoneRestaurant': 193,
 'titleProduct': 194,
 'totaltimeRecipe': 195,
 'trackMusicAlbum': 196,
 'typicalagerangeEvent': 197,
 'versionCreativeWork': 198
 #'offerdetailsProduct': 144,
# 'founderLocalBusiness': 118,
 # 'mainentityofpagePerson': 118,
 #'eventattendancemodeEvent': 201,
 #'performerEvent': 202,
 #'inalbumMusicRecording': 203,
 #'eventstatusEvent': 204
 }

In [73]:
small

,text,name,label_complete
0,Heavenly Sentiments Spray Peaceful White Lilie...,name,nameProduct
1,pricecurrency USD lowprice 202 95 highprice 2...,offers,offersProduct
2,reviewcount 4 ratingvalue 5 00 reviewcount 25...,aggregaterating,aggregateratingProduct
3,author from - description Arrangement was mad...,review,reviewProduct
4,NewCondition NewCondition NewCondition NewCond...,itemcondition,itemconditionProduct
...,...,...,...
21452,5 0 cu ft Smart Front Load Washer with Load Go...,name,nameProduct
21453,pricecurrency USD price 1349 99 availability ...,offers,offersProduct
21454,name Whirlpool name GE Appliances name GE App...,brand,brandProduct
21455,reviewcount 7 itemreviewed 5 0 cu ft Smart Fr...,aggregaterating,aggregateratingProduct


In [77]:
test

,text,label
0,Plauen Indonesien Freiburg Iraq-Kurdistan Boch...,130.0
1,2022-03-27T16 30 00 01 00 2020-11-15T22 00 00 ...,68.0
2,name Abenteuer Arena name Paulussaal name Deu...,115.0
3,name Freitr umer Festival name MUNDOlogia nam...,153.0
4,M y l m kem Sunny Way RB1119A T M T 2 C NH K N...,136.0
...,...,...
48530,name Virtually name Virtually name Virtually ...,115.0
48531,email c 104 i 108 100 114 101 n 115 st 97 f 1...,153.0
48532,Marina Olivier Celia Tatjana Valentine Gaia R ...,83.0
48533,Shunina de LAGARDE van der Beek Mavrenko BOITE...,71.0


In [76]:
test['label'] = test.label_complete.map(labels_new)

test.drop(['label_complete', 'name'], axis=1, inplace=True)
test

AttributeError: 'DataFrame' object has no attribute 'label_complete'

In [78]:
test[test['label'].isna()]

,text,label
1165,jobtitle Manager telephone 918037405009 name ...,NaN
2362,11 49 Buy Now 23 82 Buy Now 20 00 Buy Now 14 ...,NaN
10957,Sean P Holman is a long-time automotive enthu...,NaN
17558,BIZE G rald SZYMANSKI Stephanie BERTRAND Marie...,NaN
19888,http www angelina-jolie com https www instagra...,NaN
23002,price 5 00 pricecurrency GBP availability in ...,NaN
23702,https cellculturedish com experts dr-juan-hou ...,NaN
45509,currency EUR seller perapasha price 2 400035 ...,NaN
45580,description Meet Angela Watkins Rivers our va...,NaN


In [79]:
test = test[~test['label'].isna()]

In [80]:
test['label'] = test.label.astype(int)

/tmp/ipykernel_725051/2755059116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'] = test.label.astype(int)


In [81]:
test

,text,label
0,Plauen Indonesien Freiburg Iraq-Kurdistan Boch...,130
1,2022-03-27T16 30 00 01 00 2020-11-15T22 00 00 ...,68
2,name Abenteuer Arena name Paulussaal name Deu...,115
3,name Freitr umer Festival name MUNDOlogia nam...,153
4,M y l m kem Sunny Way RB1119A T M T 2 C NH K N...,136
...,...,...
48530,name Virtually name Virtually name Virtually ...,115
48531,email c 104 i 108 100 114 101 n 115 st 97 f 1...,153
48532,Marina Olivier Celia Tatjana Valentine Gaia R ...,83
48533,Shunina de LAGARDE van der Beek Mavrenko BOITE...,71


In [82]:
small['label'] = small.label_complete.map(labels_new)

small.drop(['label_complete', 'name'], axis=1, inplace=True)
small

,text,label
0,Heavenly Sentiments Spray Peaceful White Lilie...,136.0
1,pricecurrency USD lowprice 202 95 highprice 2...,147.0
2,reviewcount 4 ratingvalue 5 00 reviewcount 25...,16.0
3,author from - description Arrangement was mad...,177.0
4,NewCondition NewCondition NewCondition NewCond...,108.0
...,...,...
21452,5 0 cu ft Smart Front Load Washer with Load Go...,136.0
21453,pricecurrency USD price 1349 99 availability ...,147.0
21454,name Whirlpool name GE Appliances name GE App...,34.0
21455,reviewcount 7 itemreviewed 5 0 cu ft Smart Fr...,16.0


In [85]:
small = small[~small['label'].isna()]

In [86]:
small['label'] = small.label.astype(int)

/tmp/ipykernel_725051/1106728590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['label'] = small.label.astype(int)


In [87]:
test['label'].unique()

array([130,  68, 115, 153, 136, 147,  16, 127,   7,  81,  34,  38, 108,
        85,  88,  19, 122,  42, 126, 175,  40, 177, 191,  67, 137, 173,
        27, 174, 172, 160,  46,  58, 195, 170, 171,  17, 112, 143, 120,
       105,  98, 134,  64,  37,  91,  25,  55,  52, 129, 167, 117,  23,
        30,  83,  71,  59,   2, 102, 131,   4, 189, 163,  14, 119, 118,
        84, 156,  94,  60, 186, 161, 194,  26, 132,   5, 190, 164,  15,
       150, 155,   3, 110,  70,  73,  97,  11, 182, 133, 196, 142, 158,
        74,  93, 140, 144,  95, 113, 121,  87,  66, 181,  35,  96,  62,
         9,  10, 187, 159,  39,  72,  86,  28, 135,  80, 192, 106, 107,
        36,  41,  77,  51,  50,  20,  13, 100,  47,  48, 188, 178,   6,
       162,  44,  45, 138,   8, 193, 184,  82, 165,  18,   1, 149, 185,
       128,  24, 104, 166,  54,  75,  22,  79,  43, 114, 103, 148, 176,
       139, 154,  69, 197, 157, 179, 169, 109,  29,  61,  78,  76, 146,
       151, 152,  53, 168,   0, 198, 183,  65,  31, 141,  56, 12

In [88]:
test[test['label'].isna()]

,text,label


In [58]:
len (test['label'].unique())

202

In [71]:
test.to_csv('../../src/data/schemafiltereddata/TrainTestTables/CSV/large_tables_prepped.csv', index=False)

In [61]:
#large = pd.read_csv('../../src/data/schemafiltereddata/TrainTestTables/CSV/large_tables_prepped.csv')
testset= pd.read_csv('../../src/data/schemafiltereddata/TrainTestTables/archiv/test_tables_9000_prepped.csv')
testset

,Unnamed: 0,text,label
0,0,Comedy Improv at Arts Garage 2020-12-10 Shows ...,130
1,1,2020-12-10T21 30 00-05 00 2020-12-10T21 30 00-...,68
2,2,name Arts Garage name Bob Carter 8217 s Actor...,115
3,3,name Mod 27 Improv telephone 561-223-1651 ema...,153
4,4,My Front Door Lord Have Mercy Sold Out Wedding...,134
...,...,...,...
49599,49599,url https eutrade-shop com epages f7e4b224-a2...,147
49600,49600,4 5 5 3 5 5 5 4 3 5 3 5 5 5 3 5 1 5 5 4 3 3 5 ...,38
49601,49601,http schema org NewCondition http schema org N...,108
49602,49602,4313042590504 0 4313042595028 0 4313042595066 ...,85


In [64]:
testset = testset.drop(columns={'Unnamed: 0'})
uniques = testset['label'].unique()

In [66]:
larges = (large['label'].unique())
uniques

array([130,  68, 115, 153, 134,  64,  37, 137, 173,  27, 174, 160,  46,
       170,  17, 112,  98, 172,  58, 195, 171, 143, 120, 105,  91,  25,
       103, 132,   5,   9,  10, 187, 159,  39, 136, 147,  38, 108, 106,
        34, 127, 200,  16, 177, 190,  79, 164,  66, 148,  21,  72, 175,
        40, 122, 126, 110,  67,  74, 135,  80, 192,  84,  55,  52, 129,
       167, 117,  15, 176, 131,   4, 189,  78, 138,   8, 193, 184,  82,
       165,  86,  41,  96,  85, 150, 191, 139,  69,   6, 102, 161, 162,
        50,  20,  87, 128,  24, 166,  54,  99,  75, 158, 188,  30,  83,
        71,  59,   2,  88,  70, 182, 119,  94,  43, 114, 156,  60, 154,
       100, 185, 197, 157, 181, 104, 141,  33,  93, 140, 199,  95, 113,
       121,  19,  11, 194,   3,  42,   7,  81, 180,  31,  22,  73, 118,
        90, 149, 123,  14, 107, 198, 163, 179, 169,  29,   0, 178, 133,
        36,  53,  92,  97, 168, 201,  49,  44, 124,  56, 145,  45,  26,
        57,  89, 196,  23, 186,  61, 116,  77,  51,  35,   1, 15

In [69]:
set(uniques)-set(larges)

set()

In [14]:
small = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/archiv/small_train_prepped.csv')

In [15]:
small

,text,label
0,Heavenly Sentiments Spray Peaceful White Lilie...,136
1,pricecurrency USD lowprice 202 95 highprice 2...,147
2,reviewcount 4 ratingvalue 5 00 reviewcount 25...,16
3,author from - description Arrangement was mad...,177
4,NewCondition NewCondition NewCondition NewCond...,108
...,...,...
13231,name - address address name name - address na...,166
13232,pricecurrency RUB availability http schema or...,145
13233,2011 0 2016 0 2006 0 2018 0 2016 0 2020 0 2013...,54
13234,736 140 560 208 208 32 288 576 272 264 286 229...,141


In [17]:
test = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/test_tables_9000_prepped.csv')

In [18]:
test

,Unnamed: 0,text,label
0,0,Comedy Improv at Arts Garage 2020-12-10 Shows ...,130
1,1,2020-12-10T21 30 00-05 00 2020-12-10T21 30 00-...,68
2,2,name Arts Garage name Bob Carter 8217 s Actor...,115
3,3,name Mod 27 Improv telephone 561-223-1651 ema...,153
4,4,My Front Door Lord Have Mercy Sold Out Wedding...,134
...,...,...,...
49599,49599,url https eutrade-shop com epages f7e4b224-a2...,147
49600,49600,4 5 5 3 5 5 5 4 3 5 3 5 5 5 3 5 1 5 5 4 3 3 5 ...,38
49601,49601,http schema org NewCondition http schema org N...,108
49602,49602,4313042590504 0 4313042595028 0 4313042595066 ...,85


In [89]:
sm_list = small['label'].unique()

In [90]:
test_list = test['label'].unique()

In [48]:
len(small['label'].unique())

201

In [91]:
set(test_list) - set(sm_list)

set()

In [50]:
set(sm_list) - set(test_list)

set()

In [51]:
small.to_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/small_train_prepped_new.csv')

In [92]:
small.to_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/medium_train_prepped_new.csv')

In [53]:
test.to_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/test_prepped_new.csv')

In [93]:
test.to_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/test_medium_prepped_new.csv')